<a href="https://colab.research.google.com/github/OlegKret/---/blob/master/%22MODIFI_ver1_1_ipynb%20removed%20column%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
from fractions import Fraction  # Import Fraction for handling fractional values
from scipy.optimize import linprog
import logging



def simplex_method_mixed(tableau, is_maximization, all_slack_vars,
                         objective_coeffs, constraint_coeffs,
                         constraint_types, b_values, M=1e6):
    """
    Performs the simplex method with the Big-M technique for problems with mixed constraints.

    Args:
        tableau: The initial tableau representing the linear programming problem.
        is_maximization: A boolean value indicating whether it's a maximization or minimization problem.
        all_slack_vars: A list of all slack/surplus/artificial variables.
        objective_coeffs: The original objective function coefficients.
        constraint_coeffs: The original constraint coefficients.
        constraint_types: The original constraint types ('<=', '>=', or '=').
        b_values: The original RHS values of the constraints.
        M: The Big-M constant (default: 1e6).

    Returns:
        A tuple containing the optimal solution (as a NumPy array) and the optimal value
        of the objective function. If the problem is unbounded or infeasible, it returns
        (None, None).
    """

    iteration = 0
    tolerance = 1e-6
    total_num_cols = tableau.shape[1] - 1

    num_vars = tableau.shape[1] - len(all_slack_vars) - 1  # Number of original decision variables

    num_vars = tableau.shape[1] - len(all_slack_vars) - 1
    num_artificial_vars = sum(1 for t in constraint_types if t in ('>=', '='))

    non_basic_artificial_vars = []
    basic_vars = [f"a{i+1}" for i in range(num_artificial_vars)]
    var_to_col = {var: i for i, var in enumerate([f'x{j+1}' for j in range(num_vars)] + all_slack_vars)}

    entering_col = None
    leaving_row = None

    # Main Simplex Loop
    while True:
        # Update objective row coefficients based on current basic variables
        # Update objective row coefficients based on current basic variables
        for col_idx in range(tableau.shape[1] - 1):
            new_coefficient = 0
            if col_idx < num_vars:  # Decision variable
                if col_idx < len(objective_coeffs):
                    new_coefficient = objective_coeffs[col_idx]

            # Iterate over valid basic_vars entries only
            for row_idx, basic_var in enumerate(basic_vars):
                if basic_var:  # Check if basic_var is not None
                    ci = 0
                    if basic_var.startswith('x'):
                        ci = objective_coeffs[int(basic_var[1:]) - 1]
                    elif basic_var.startswith('a'):
                        ci = M

                    new_coefficient -= tableau[row_idx, col_idx] * ci

            tableau[-1, col_idx] = new_coefficient

        # Calculate RHS of the objective row
        # Calculate RHS of the objective row based on Ci and Bi
        tableau[-1, -1] = sum(objective_coeffs[int(var[1:]) - 1] * tableau[i, -1]
                              for i, var in enumerate(basic_vars) if var.startswith('x'))

        # Set coefficients of artificial variables in the objective row to 0
        for var in all_slack_vars:
            if var.startswith('a'):
                tableau[-1, var_to_col[var]] = 0

        print(f"\nIteration {iteration}:")
        print_tableau_as_fractions(tableau)
        print_tableau_with_key_info(tableau, entering_col, leaving_row, basic_vars, objective_coeffs, all_slack_vars, is_maximization, iteration, M, var_to_col, num_vars, constraint_types)

        # 1. Check for optimality
        if is_optimal(tableau, is_maximization):
            # Check if any artificial variables are still in the basis with positive values
            if any(var.startswith('a') and tableau[i, -1] > tolerance
                   for i, var in enumerate(basic_vars)):
                print("Infeasible solution! Artificial variables are still in the basis with positive values.")
                return None, None  # Infeasible
            else:
                print("Optimal solution found!")
                break

        # 2. Identify entering variable
        entering_col = find_entering_variable(tableau, is_maximization)
        if entering_col is None:
            print("Optimal solution found!")
            break

        print(f"--> Entering column: {entering_col}")

        # 3. Identify leaving variable
        leaving_row = find_leaving_variable(tableau, entering_col, all_slack_vars,
                                            non_basic_artificial_vars, num_vars, basic_vars)

        if leaving_row is None:
            # Check for unboundedness only if no artificial variables are in the basis
            if not any(var.startswith('a') for var in basic_vars) and \
               not check_for_unboundedness(tableau, num_vars, all_slack_vars):
                return None, None  # Unbounded
            else:
                print("All ratios are infinite or involve non-basic artificial variables. Continuing to the next iteration.")
        else:
            print(f"--> Leaving row: {leaving_row}")
            leaving_var = basic_vars[leaving_row]

            if leaving_var.startswith('a'):
                print(f"--> Artificial variable {leaving_var} is leaving the basis")
                perform_pivot_and_remove_artificial(
                    tableau, leaving_row, entering_col, leaving_var, all_slack_vars, basic_vars  # Pass basic_vars here
                )
                non_basic_artificial_vars.append(leaving_var)
                total_num_cols = tableau.shape[1] - 1

                # Update var_to_col after removing the artificial variable
                var_to_col = {var: i for i, var in enumerate([f'x{j+1}' for j in range(tableau.shape[1] - len(all_slack_vars) - 1)] + all_slack_vars)}

                # If leaving_row points to the last row (which was removed), reset it
                if leaving_row == tableau.shape[0]:
                    leaving_row = None

            # Perform pivot operation
            # Perform pivot operation
            if leaving_row is not None:
                perform_pivot(tableau, leaving_row, entering_col, iteration,
                              basic_vars, objective_coeffs, all_slack_vars, num_vars, var_to_col, M)

                # Handle potential changes in entering_col and leaving_row after pivot
                if entering_col >= tableau.shape[1] - 1:  # If entering_col was removed
                    entering_col = find_entering_variable(tableau, is_maximization)
                    if entering_col is not None:
                        leaving_row = find_leaving_variable(tableau, entering_col, all_slack_vars,
                                                            non_basic_artificial_vars, num_vars, basic_vars)
                    else:
                        leaving_row = None

        iteration += 1



        # Pause after each iteration
        input("Press Enter to continue to the next iteration, or type 'quit' to stop: ")
        if input().lower() == 'quit':
            break

        # Check and remove any non-basic artificial variable columns
        tableau, num_vars, all_slack_vars, entering_col, leaving_row_update = \
            remove_non_basic_artificial_vars(tableau, num_vars, all_slack_vars, entering_col, non_basic_artificial_vars, basic_vars)

        # Update leaving_row only if it's valid after potential changes
        if leaving_row_update is not None and leaving_row_update < tableau.shape[0] - 1:
            leaving_row = leaving_row_update

        # If all artificial variables are removed and no leaving variable found, check again
        if not any(var.startswith('a') for var in all_slack_vars) and leaving_row is None:
            if not handle_no_leaving_variable(tableau, num_vars, all_slack_vars, is_maximization, basic_vars):
                return None, None


        # Update objective row coefficients based on current basic variables
        for col_idx in range(tableau.shape[1] - 1):
            new_coefficient = 0
            if col_idx < num_vars:  # Decision variable
                if col_idx < len(objective_coeffs):
                    new_coefficient = objective_coeffs[col_idx]

            for row_idx, basic_var in enumerate(basic_vars):
                if basic_var and basic_var.startswith('x'):
                    var_index = int(basic_var[1:]) - 1
                    # Check if the variable index is valid and corresponds to an original variable
                    if 0 <= var_index < num_vars and var_index < len(objective_coeffs):
                        ci = objective_coeffs[var_index]

                        slack_var_index = col_idx - num_vars
                        if 0 <= slack_var_index < len(all_slack_vars):
                            new_coefficient -= tableau[row_idx, col_idx] * ci

            tableau[-1, col_idx] = new_coefficient

        # Calculate RHS of the objective row, handling None in basic_vars
        tableau[-1, -1] = sum(
            objective_coeffs[int(var[1:]) - 1] * tableau[i, -1]
            for i, var in enumerate(basic_vars)
            if var is not None and var.startswith('x') and 0 <= int(var[1:]) - 1 < len(objective_coeffs)
        )

        # Debugging Enhancements: Print key variables after each iteration
        print(f"\n--- Debugging Information (Iteration {iteration}) ---")
        print("basic_vars:", basic_vars)
        print("all_slack_vars:", all_slack_vars)
        print("num_vars:", num_vars)
        print("var_to_col:", var_to_col)
        print("---------------------------------------------------\n")

        print(f"\nIteration {iteration}:")
        print_tableau_as_fractions(tableau)
        print_tableau_with_key_info(tableau, entering_col, leaving_row, basic_vars, objective_coeffs, all_slack_vars, is_maximization, iteration, M, var_to_col, num_vars, constraint_types)

    # Extract and print the optimal solution and objective value
    optimal_solution = extract_optimal_solution(tableau, num_vars, all_slack_vars)
    optimal_value = calculate_optimal_value(optimal_solution, objective_coeffs)

    print("\nFinal Tableau:")
    print_tableau_as_fractions(tableau)
    print("\nOptimal solution:")
    for i in range(num_vars):
        print(f"x{i+1} = {optimal_solution[i]}")
    print("Optimal value:", optimal_value)

    return optimal_solution, optimal_value



def print_tableau_with_key_info(tableau, entering_col, leaving_row, basic_vars,
                                objective_coeffs, all_slack_vars, is_maximization,
                                iteration, M, var_to_col, num_x_vars, constraint_types):
    """
    Prints the tableau with elements represented as fractions and includes
    Key Column, Key Ratio, Ci, and Bi columns. It also handles the potential
    IndexError when accessing `objective_coeffs` and ensures correct alignment
    even after artificial variable removal.

    Args:
        tableau: The current tableau (NumPy array).
        entering_col: The index of the entering column (or None if not applicable).
        leaving_row: The index of the leaving row (or None if not applicable).
        basic_vars: A list of basic variables for each row.
        objective_coeffs: The original coefficients of the objective function.
        all_slack_vars: A list of all slack/surplus/artificial variables.
        is_maximization: True if it's a maximization problem, False otherwise.
        iteration: The current iteration number.
        M: The Big-M constant used in the Big-M method.
        var_to_col: A dictionary mapping variable names to their column indices in the tableau.
    """

    # Inside the function, you can now use constraint_types
    num_constraints = len(constraint_types)

    # Convert tableau elements to fractions for display
    fraction_tableau = np.vectorize(lambda x: Fraction(x).limit_denominator())(tableau.copy())

    # Create the header row with variable names and additional columns
    num_decision_vars = tableau.shape[1] - len(all_slack_vars) - 1
    # Header with variable names
    header = [f"x{i + 1}" for i in range(num_x_vars)] + all_slack_vars + ["RHS", "Key Column", "Key Ratio", "Ci", "Bi"]
    print(header)

    # Add a row showing the original coefficients of the objective function
    orig_coeffs_row = (["Orig"] + objective_coeffs +
                       [0] * len(constraint_types) +
                       [M if var.startswith('a') else 0 for var in all_slack_vars])
    print(orig_coeffs_row)

    # Iterate through each row of the tableau
    for row_index, row in enumerate(fraction_tableau):
        # Format the row elements as fractions (numerator/denominator)
        formatted_row = [f"{frac.numerator}/{frac.denominator}" for frac in row]

        # Insert the basic variable name at the beginning of the row
        if row_index < len(basic_vars) and basic_vars[row_index] is not None:
            formatted_row.insert(0, basic_vars[row_index])
        else:
            formatted_row.insert(0, 'Z')

        # Add Key Column and Key Ratio information if applicable
        if row_index == leaving_row:
            formatted_row.append(entering_col)
            if entering_col is not None and tableau[row_index, entering_col] > 0:
                key_ratio = tableau[row_index, -1] / tableau[row_index, entering_col]
                formatted_row.append(f"{Fraction(key_ratio).limit_denominator()}")
            else:
                formatted_row.append("-")
        else:
            formatted_row.extend(["-", "-"])

        # Initialize calc_row even for non-objective rows
        calc_row = []

        # Add Ci (coefficient in objective function) and Bi (basic variable name)
        # Add Ci (coefficient in objective function) and Bi (basic variable name)
        if row_index < len(basic_vars) and basic_vars[row_index] is not None:
            basic_var = basic_vars[row_index]
            ci_col = var_to_col.get(basic_var, None)
            if ci_col is not None:
                if basic_var.startswith('a'):
                    ci = 'M' if not is_maximization else '-M'
                elif basic_var.startswith('x') and 0 <= ci_col < len(objective_coeffs): # Added bounds check
                    ci = objective_coeffs[ci_col]
                else:
                    ci = 0
            else:
                ci = "-"
            formatted_row.extend([ci, basic_var])
        else:  # Objective row or constraint row without a basic variable
            # Add a row to display calculation results for the objective row
            # Add a row to display calculation results for the objective row
            calc_row = ["Calc"]
            for col_idx in range(tableau.shape[1] - 1):
                calc_str = ""
                if col_idx < num_x_vars:
                    if col_idx < len(objective_coeffs):
                        calc_str += f"{orig_coeffs_row[col_idx + 1]} - ("
                    else:
                        calc_str += "0 - ("
                elif col_idx < num_x_vars + num_constraints:
                    calc_str += f"{orig_coeffs_row[col_idx + 1]} - ("

                # Iterate over all rows with basic variables
                for row_idx, basic_var in enumerate(basic_vars):
                    if basic_var:
                        ci = 0
                        if basic_var.startswith('x'):
                            var_index = int(basic_var[1:]) - 1
                            if 0 <= var_index < len(objective_coeffs): # Bounds check
                                ci = objective_coeffs[var_index]
                        elif basic_var.startswith('a'):
                            ci = M

                        calc_str += f"{tableau[row_idx, col_idx]} * {ci} + "

                calc_str = calc_str[:-3] + ")"  # Remove the last " + "

                if col_idx < num_x_vars + len(constraint_types):
                    calc_row.append(calc_str)
                else:
                    calc_row.append('0')

            # Add RHS calculation to calc_row
            rhs_calc = " + ".join([f"({tableau[i, -1]} * {M})"
                                   for i, basic_var in enumerate(basic_vars) # Iterate with enumerate
                                   if basic_var and basic_var.startswith('a')])
            calc_row.append(rhs_calc)
            print(calc_row)

            # In the first iteration, update the objective row in the tableau
            # to match the calculated values
            # In the first iteration, update the objective row in the tableau to match the calculated values
            if iteration == 0:
                for j in range(1, len(calc_row)):
                    tableau[-1, j - 1] = eval(calc_row[j])


            # Print the initial tableau with the corrected objective row
            if iteration == 0:
                print("\nInitial Tableau:")
                print(tableau)

            # In the first iteration, display the actual values from the tableau for the objective row
            if iteration == 0:
                for j in range(len(formatted_row) - 5):
                    formatted_row[j] = f"{tableau[-1, j]}"
            else:  # Set Ci only after the first iteration
                for j, var in enumerate([f'x{k + 1}' for k in range(num_x_vars)] + all_slack_vars):
                    if j < len(formatted_row) - 5:
                        if var.startswith('x') and (int(var[1:]) - 1) < len(objective_coeffs):
                            formatted_row[j] = objective_coeffs[int(var[1:]) - 1] if not is_maximization else -objective_coeffs[int(var[1:]) - 1]
                        elif var.startswith('a'):
                            formatted_row[j] = 'M' if not is_maximization else '-M'
                        else:  # Surplus variable
                            formatted_row[j] = '0'

            if leaving_row is not None:
                formatted_row[-1] = basic_vars[leaving_row]
            else:
                formatted_row[-1] = '-'
            formatted_row.extend(["-", "-"])

        print(formatted_row)





def create_tableau_for_maximization(num_vars, objective_coeffs, num_constraints,
                                     constraint_coeffs, constraint_types, b_values, is_maximization):
    """
    Creates the initial tableau for the simplex method, handling both maximization and minimization problems.

    Args:
        num_vars: The number of decision variables.
        objective_coeffs: Coefficients of the objective function.
        num_constraints: The number of constraints.
        constraint_coeffs: Coefficients of the constraints.
        constraint_types: Types of constraints ('<=', '>=', or '=').
        b_values: Right-hand side values of the constraints.
        is_maximization: Boolean indicating if it's a maximization problem (True) or minimization (False).

    Returns:
        tableau: The initial tableau.
        all_slack_vars: List of all slack/surplus/artificial variables.
    """

    print("Original Input:")
    print("num_vars:", num_vars)
    print("objective_coeffs:", objective_coeffs)
    print("num_constraints:", num_constraints)
    print("constraint_coeffs:", constraint_coeffs)
    print("constraint_types:", constraint_types)
    print("b_values:", b_values)

    # Ensure all RHS values are non-negative
    for i in range(num_constraints):
        if b_values[i] < 0:
            constraint_coeffs[i] = [-coeff for coeff in constraint_coeffs[i]]
            b_values[i] *= -1
            constraint_types[i] = '>=' if constraint_types[i] == '<=' else '<='

    print("\nAfter ensuring non-negative RHS:")
    print("constraint_coeffs:", constraint_coeffs)
    print("constraint_types:", constraint_types)
    print("b_values:", b_values)

    # Count the number of '>=' and '=' constraints to determine the number of artificial variables needed
    num_artificial_vars = sum(1 for t in constraint_types if t in ('>=', '='))

    # Calculate the total number of columns in the tableau
    num_extra_cols = num_constraints  # Only one slack/surplus/artificial per constraint
    tableau = np.zeros((num_constraints + 1, num_vars + num_extra_cols + 1))

    all_slack_vars = []
    basic_vars = [None] * num_constraints

    # Fill in the tableau with coefficients, adding slack/surplus/artificial variables as needed
    artificial_var_count = 0
    for i in range(num_constraints):
        for j in range(num_vars):
            tableau[i, j] = constraint_coeffs[i][j]

        if constraint_types[i] == '<=':
            tableau[i, num_vars + i] = 1  # Add slack variable
            all_slack_vars.append(f"s{i + 1}")
        elif constraint_types[i] == '>=':
            tableau[i, num_vars + i] = -1  # Add surplus variable
            tableau[i, num_vars + i] = 1  # Add artificial variable in the SAME column
            all_slack_vars.append(f"a{artificial_var_count + 1}")
            basic_vars[i] = f"a{artificial_var_count + 1}"
            artificial_var_count += 1
        else:  # '=' constraint
            tableau[i, num_vars + i] = 1  # Add artificial variable
            all_slack_vars.append(f"a{artificial_var_count + 1}")
            basic_vars[i] = f"a{artificial_var_count + 1}"
            artificial_var_count += 1

        tableau[i, -1] = b_values[i]  # Set the RHS values

    print("\nTableau after adding slack/surplus/artificial variables:")
    print(tableau)
    print("all_slack_vars:", all_slack_vars)


    num_x_vars = tableau.shape[1] - len(all_slack_vars) - 1

    M = 1e6  # Big M value

    # Set objective row coefficients based on variable types
    for col_idx in range(tableau.shape[1] - 1):
        coefficient = 0

        if col_idx < num_vars:  # Decision variable
            coefficient = objective_coeffs[col_idx]
        elif col_idx < num_vars + num_constraints:  # Slack/surplus variable
            if constraint_types[col_idx - num_vars] == '>=':  # Surplus variable
                coefficient = -M if is_maximization else 0
            else:  # Slack variable
                coefficient = 0

        # Subtract the contribution of artificial variables ONLY for the relevant rows
        for row_idx in range(num_artificial_vars):
            coefficient -= tableau[row_idx, col_idx] * M

        # Finally, negate the coefficient if it's a maximization problem
        if is_maximization:
            coefficient = -coefficient

        tableau[-1, col_idx] = coefficient

    # Calculate RHS of the objective row
    tableau[-1, -1] = sum(b_values[i] * M for i in range(num_constraints) if constraint_types[i] in ('>=', '='))

    # Map variable names to their column indices
    var_to_col = {var: i for i, var in enumerate([f'x{j+1}' for j in range(num_vars)] + all_slack_vars)}

    # Print the tableau with Ci, Bi, RHS, and KR after setting the objective row
    print("\nTableau after setting objective row and adding -M or +M for artificial variables:")
    print_tableau_with_key_info(
        tableau, None, None, basic_vars,
        objective_coeffs, all_slack_vars, is_maximization, 0, M, var_to_col, num_x_vars, constraint_types
    )

    print("\nInitial Tableau:")
    print(tableau)

    return tableau, all_slack_vars



# def perform_m_elimination(tableau, constraint_types, b_values, num_artificial_vars, num_constraints):

#     M = 1e6
#     num_vars = tableau.shape[1] - 2 * len(constraint_types) - 1
#     if num_vars < 0:
#         num_vars = tableau.shape[1] - len(constraint_types) - 1

#     # New M-elimination logic
#     artificial_vars = [i for i, type in enumerate(constraint_types) if type in ('>=', '=')]
#     for col_idx in range(tableau.shape[1] - 1):  # Iterate over all columns except RHS
#         m_sum = 0
#         for row_idx in artificial_vars:
#             m_sum += tableau[row_idx, col_idx] * M
#         if col_idx < num_vars:  # Decision variable
#             tableau[-1, col_idx] = objective_coeffs[col_idx] - m_sum
#         elif col_idx < num_vars + num_constraints and constraint_types[col_idx - num_vars] == '>=':  # Surplus variable
#             tableau[-1, col_idx] = - m_sum
#         elif col_idx >= num_vars + num_constraints: # Artificial variable
#             tableau[-1, col_idx] = 0

#     # Calculate RHS of the objective row
#     tableau[-1, -1] = sum(M * b_values[i] for i in artificial_vars)

#     return tableau




def simplex_method_leq(tableau, objective_coeffs, constraint_types):
    """
    Performs the simplex method for problems with only <= constraints.
    """

    # Assuming all constraints are '<='
    constraint_types = ['<='] * (tableau.shape[0] - 1)

    iteration = 0

    while True:
        print(f"\nIteration {iteration}:")
        print_tableau_as_fractions(tableau)

        if np.all(tableau[-1, :-1] >= 0):
            print("Optimal solution found!")
            break

        entering_col = np.argmin(tableau[-1, :-1])
        print(f"--> Entering column: {entering_col}")

        ratios = []
        for i in range(tableau.shape[0] - 1):
            if tableau[i, entering_col] > 0:
                ratios.append(tableau[i, -1] / tableau[i, entering_col])
            else:
                ratios.append(np.inf)

        if np.all(np.isinf(ratios)):
            print("Unbounded solution!")
            return None, None

        leaving_row = np.argmin(ratios)
        print(f"--> Leaving row: {leaving_row}")

        pivot_element = tableau[leaving_row, entering_col]
        print(f"--> Pivot element: {pivot_element}")

        if abs(pivot_element) < 1e-10:
            print("Error: Pivot element is too small. Potential numerical instability.")
            return None, None

        tableau[leaving_row] /= pivot_element

        for i in range(tableau.shape[0]):
            if i != leaving_row:
                tableau[i] -= tableau[leaving_row] * tableau[i, entering_col]

        iteration += 1

    # Extract and print the optimal solution
    num_vars = tableau.shape[1] - len(constraint_types) - 1
    basic_vars = [i for i in range(tableau.shape[1] - 1)
                   if np.count_nonzero(tableau[:-1, i]) == 1 and tableau[np.nonzero(tableau[:-1, i])[0][0], i] == 1]

    optimal_solution = np.zeros(num_vars)
    for var in basic_vars:
        if var < num_vars:
            optimal_solution[var] = tableau[np.nonzero(tableau[:-1, var])[0][0], -1]

    print("\nOptimal solution:")
    for i in range(num_vars):
        print(f"x{i+1} = {optimal_solution[i]}")

    optimal_value = 0
    for i in range(num_vars):
        if i in basic_vars:
            optimal_value += objective_coeffs[i] * optimal_solution[i]

    print("Optimal value:", optimal_value)

    return optimal_solution, optimal_value




def check_for_unboundedness(tableau, num_vars, all_slack_vars, exclude_artificial=False):
    """
    Checks for unboundedness in the tableau, considering the presence of artificial variables.

    Args:
        tableau: The current tableau.
        num_vars: The number of original decision variables.
        all_slack_vars: A list of all slack/surplus/artificial variables.
        exclude_artificial: Whether to exclude artificial variables from the unboundedness check (default: False).

    Returns:
        True if the problem is not unbounded, False if it is unbounded.
    """

    tolerance = 1e-6

    print(f"--> Checking for unboundedness with num_vars = {num_vars}, exclude_artificial={exclude_artificial}")

    # Calculate the number of original variables remaining in the tableau
    num_original_vars = num_vars - sum(1 for var in all_slack_vars if var.startswith('s'))

    for col in range(tableau.shape[1] - 1):  # Exclude the RHS column
        is_original_var = col < num_original_vars

        # Adjust indexing into all_slack_vars after artificial variable removal
        slack_surplus_var_index = col - num_original_vars
        if 0 <= slack_surplus_var_index < len(all_slack_vars):
            is_slack_or_surplus_var = all_slack_vars[slack_surplus_var_index].startswith('s')

            if exclude_artificial:
                is_artificial_var = all_slack_vars[slack_surplus_var_index].startswith('a')
            else:
                is_artificial_var = False
        else:
            is_slack_or_surplus_var = False
            is_artificial_var = False

        # Print relevant variables and conditions
        print(f"--> Column {col}:")
        print(f"    is_original_var={is_original_var}")
        print(f"    is_slack_or_surplus_var={is_slack_or_surplus_var}")
        print(f"    is_artificial_var={is_artificial_var}")
        print(f"    tableau[-1, col]={tableau[-1, col]}")  # Objective row coefficient
        print(f"    tableau[:-1, col]={tableau[:-1, col]}")  # Constraint row coefficients

        all_constraints_non_positive = np.all(tableau[:-1, col] <= tolerance)
        print(f"    np.all(tableau[:-1, col] <= tolerance)={all_constraints_non_positive}")

        if (
            is_original_var  # Only consider original variables
            and not is_artificial_var
            and not is_slack_or_surplus_var
            and tableau[-1, col] < 0  # Check for negative coefficient in the objective row (for maximization)
            and all_constraints_non_positive
        ):
            print("--> Unboundedness detected!")
            return False

    return True  # No unboundedness detected

def print_tableau_as_fractions(tableau):
    """
    Prints the tableau with elements represented as fractions in the format 'numerator/denominator'.
    """
    fraction_tableau = np.vectorize(lambda x: Fraction(x).limit_denominator())(tableau)
    for row in fraction_tableau:
        formatted_row = [f"{frac.numerator}/{frac.denominator}" for frac in row]
        print(formatted_row)





def extract_optimal_solution(tableau, num_vars, all_slack_vars):
    """
    Extracts the optimal solution from the final tableau, considering the presence of
    slack/surplus/artificial variables.

    Args:
        tableau: The final tableau after the simplex algorithm has terminated.
        num_vars: The number of original decision variables in the problem.
        all_slack_vars: A list containing the names of all slack/surplus/artificial variables.

    Returns:
        A NumPy array representing the optimal solution, containing values only for the original
        decision variables. If the problem is infeasible, it returns None.
    """

    # Identify basic variables (columns with a single 1 and other elements 0)
    basic_var_cols = [i for i in range(tableau.shape[1] - 1) if
                    np.count_nonzero(tableau[:-1, i]) == 1 and
                    tableau[np.nonzero(tableau[:-1, i])[0][0], i] == 1]

    print(f"Basic variables (column indices): {basic_var_cols}")

    # Initialize the optimal solution array
    optimal_solution = np.zeros(num_vars)

    # Check if any artificial variables remain in the basis (indicates infeasibility)
    if any(var.startswith('a') and
         all(np.isclose(tableau[:-1, all_slack_vars.index(var) + num_vars], 0, atol=1e-6))
         for var in all_slack_vars):
      print("Infeasible solution! Artificial variables are still in the basis.")
      return None

    for var_index in basic_var_cols:
        if var_index < num_vars:
            basic_row = np.nonzero(tableau[:-1, var_index])[0][0]
            # Adjust column indexing for RHS
            rhs_col = tableau.shape[1] - 1
            optimal_solution[var_index] = tableau[basic_row, rhs_col]
            print(f"Variable x{var_index + 1} is basic in row {basic_row}, value = {tableau[basic_row, -1]}")

    return optimal_solution




def calculate_optimal_value(optimal_solution, objective_coeffs):
    """
    Calculates the optimal value of the objective function.
    """
    optimal_value = 0

    # Iterate only over the original variables (length of objective_coeffs)
    for i in range(len(objective_coeffs)):
        optimal_value += objective_coeffs[i] * optimal_solution[i]

    return optimal_value







def is_optimal(tableau, is_maximization):
    """
    Checks if the current tableau represents an optimal solution.
    """

    print("Objective row in is_optimal:", tableau[-1, :])

    if is_maximization:
        # Check if all coefficients (including RHS) are non-negative
        result = np.all(tableau[-1, :] >= 0)
    else:
        # Check if all coefficients (including RHS) are non-positive
        result = np.all(tableau[-1, :] <= 0)

    print("is_optimal result:", result)
    return result


def find_entering_variable(tableau, is_maximization):
    """
    Identifies the entering variable using the new logic:
    - Column with the most negative value in the objective row
    """

    # Find the column index with the most negative value in the objective row
    entering_col = np.argmin(tableau[-1, :-1])

    # If all values are non-negative (for minimization, converted to maximization), it's optimal
    if tableau[-1, entering_col] >= 0:
        return None

    return entering_col


def find_leaving_variable(tableau, entering_col, all_slack_vars, non_basic_artificial_vars, num_vars, basic_vars):
    """
    Finds the leaving variable based on the minimum ratio test,
    excluding non-basic artificial variables.
    """

    print(f"--> In find_leaving_variable: all_slack_vars={all_slack_vars}, non_basic_artificial_vars={non_basic_artificial_vars}, basic_vars={basic_vars}")

    ratios = []
    valid_rows = []  # Keep track of valid rows

    for row_idx in range(tableau.shape[0] - 1):  # Exclude the objective row
        # Get the basic variable for this row
        basic_var = basic_vars[row_idx]

        # Check if the basic variable is a non-basic artificial variable
        is_non_basic_artificial = basic_var in non_basic_artificial_vars

        if is_non_basic_artificial:
            print(f"--> Skipping row {row_idx} because basic variable {basic_var} is a non-basic artificial variable")
            continue  # Skip this row

        valid_rows.append(row_idx)

        if tableau[row_idx, entering_col] > 0:
            # Adjust column indexing for RHS
            rhs_col = tableau.shape[1] - 1
            ratios.append(tableau[row_idx, rhs_col] / tableau[row_idx, entering_col])
        else:
            ratios.append(np.inf)

    print(f"--> Ratios: {ratios}")

    if np.all(np.isinf(ratios)):
        return None

    # Find the minimum ratio among valid rows only
    min_ratio_index = np.argmin(ratios)
    leaving_row = valid_rows[min_ratio_index]

    return leaving_row



def handle_no_leaving_variable(tableau, num_vars, all_slack_vars, is_maximization, basic_vars):
    """
    Handles the case where no leaving variable is found.

    Modified to correctly handle the case where no leaving variable is found
    due to all non-positive coefficients in the entering column, even with
    artificial variables present.
    """

    # Check if any artificial variables are still in the basis
    if any(var.startswith('a') for var in basic_vars):
        print("All ratios are infinite, but artificial variables are still present. Continue to the next iteration.")
        return True  # Continue if artificial variables are present

    # If not unbounded and no artificial variables, it's optimal
    if is_maximization:
        if np.all(tableau[-1, :-1] >= 0):
            print("Optimal solution found!")
    else:  # Minimization (converted to maximization)
        if np.all(tableau[-1, :-1] <= 0):
            print("Optimal solution found!")

    return True  # Optimal or continue if no artificial variables in basis


def perform_pivot_and_remove_artificial(tableau, leaving_row, entering_col, leaving_var, all_slack_vars, basic_vars):
    """
    Performs pivot operation and zeroes out the artificial variable column.
    """

    pivot_element = tableau[leaving_row, entering_col]
    print(f"--> Pivot element: {pivot_element}")

    if abs(pivot_element) < 1e-10:
        print("Error: Pivot element is too small. Potential numerical instability.")
        return

    # Perform pivot operation
    tableau[leaving_row] /= pivot_element
    for i in range(tableau.shape[0]):
        if i != leaving_row:
            tableau[i] -= tableau[leaving_row] * tableau[i, entering_col]

    # Zero out the entire column corresponding to the leaving artificial variable EXCEPT the pivot row
    leaving_var_col = all_slack_vars.index(leaving_var) + tableau.shape[1] - len(all_slack_vars) - 1
    tableau[:, leaving_var_col] = 0
    tableau[leaving_row, leaving_var_col] = 1

    # Update basic_vars
    basic_vars[leaving_row] = None
    print(f"--> Artificial variable {leaving_var} left the basis (column zeroed out)")

    # Recalculate num_vars after zeroing out the column
    num_vars = tableau.shape[1] - len(all_slack_vars) - 1

    # Check if any original or slack/surplus variable columns are all zero (except for the objective row)
    for col_idx in range(num_vars + len(all_slack_vars)):
        if np.all(np.isclose(tableau[:-1, col_idx], 0, atol=1e-6)):
            # Find the row where this variable was basic (if any)
            basic_row = next((i for i, var in enumerate(basic_vars)
                              if var == (all_slack_vars[col_idx - num_vars] if col_idx >= num_vars else f'x{col_idx + 1}')),
                             None)
            if basic_row is not None:
                basic_vars[basic_row] = None
                var_name = all_slack_vars[col_idx - num_vars] if col_idx >= num_vars else f'x{col_idx + 1}'
                print(f"--> Variable {var_name} is no longer basic (column zeroed out)")

    print(f"--> Updated basic_vars after checking for zeroed columns: {basic_vars}")




def perform_pivot(tableau, leaving_row, entering_col, iteration,
                  basic_vars, objective_coeffs, all_slack_vars, num_vars, var_to_col, M):
    """
    Performs the standard pivot operation, updates the objective row, and
    removes artificial variables if they leave the basis.
    """

    pivot_element = tableau[leaving_row, entering_col]
    print(f"--> Pivot element: {pivot_element}")

    if abs(pivot_element) < 1e-10:
        print("Error: Pivot element is too small. Potential numerical instability.")
        return

    # Assign leaving_var here, before the if-else block
    leaving_var = basic_vars[leaving_row]

    # 1. Update key row (Divide the pivot row by the pivot element)
    tableau[leaving_row] /= pivot_element

    # 2. Update non-key rows
    for i in range(tableau.shape[0]):
        if i != leaving_row:
            if iteration > 0 or i == tableau.shape[0] - 1:
                key_ratio = tableau[i, entering_col] / pivot_element
                tableau[i] -= tableau[leaving_row] * key_ratio

    # 3. Update the objective row coefficient for the entering variable
    entering_var = all_slack_vars[entering_col - num_vars] if entering_col >= num_vars else f'x{entering_col + 1}'

    if entering_var.startswith('x'):
        var_index = int(entering_var[1:]) - 1
        if 0 <= var_index < len(objective_coeffs):
            tableau[-1, entering_col] = objective_coeffs[var_index]
    else:
        tableau[-1, entering_col] = 0

    # 4. Recalculate the objective row (Zj - Cj)
    for col_idx in range(tableau.shape[1] - 1):
        new_coefficient = 0
        for row_idx in range(tableau.shape[0] - 1):
            basic_var = basic_vars[row_idx]
            if basic_var:
                ci = 0
                if basic_var.startswith('x'):
                    ci = objective_coeffs[int(basic_var[1:]) - 1]

                new_coefficient -= tableau[row_idx, col_idx] * ci

        tableau[-1, col_idx] = new_coefficient

    # 5. Recalculate RHS of the objective row
    tableau[-1, -1] = sum(
        objective_coeffs[int(var[1:]) - 1] * tableau[i, -1]
        for i, var in enumerate(basic_vars) if var and var.startswith('x')
    )

    print(f"\nTableau after pivot operation (Iteration {iteration}):")
    print_tableau_as_fractions(tableau)

    # 6. Remove any artificial variable column that has left the basis
    if leaving_var and leaving_var.startswith('a'):
        # ... (removal of artificial variable column and update of var_to_col remains the same)

        # Update basic_vars after removing the artificial variable
        basic_vars[leaving_row] = None

        # Update num_vars after removing an artificial variable
        num_vars = tableau.shape[1] - len(all_slack_vars) - 1

        print(f"--> Removed artificial variable column {leaving_var}")

    else:
        # Update basic_vars
        entering_var = all_slack_vars[entering_col - num_vars] if entering_col >= num_vars else f'x{entering_col + 1}'
        basic_vars[leaving_row] = entering_var

        print(f"--> Updated basic_vars: {basic_vars}")


def remove_non_basic_artificial_vars(tableau, num_vars, all_slack_vars, entering_col, non_basic_artificial_vars, basic_vars):
    """
    Checks and removes non-basic artificial variables.
    """
    leaving_row = None

    for var in all_slack_vars[:]:  # Iterate over a copy to avoid modification during iteration
        if var.startswith('a'):
            col_index = all_slack_vars.index(var) + num_vars
            if all(np.isclose(tableau[:-1, col_index], 0, atol=1e-6)):

                tableau = np.delete(tableau, col_index, axis=1)
                all_slack_vars.remove(var)
                non_basic_artificial_vars.append(var)

                # Remove the variable from basic_vars if it's present
                for i in range(len(basic_vars)):
                    if basic_vars[i] == var:
                        basic_vars[i] = None

                print(f"--> Removed artificial variable column {var} (became non-basic)")

                # If entering_col was removed, reselect entering_col and leaving_row
                if entering_col >= tableau.shape[1] - 1:
                    entering_col = np.argmin(tableau[-1, :-1])

                    # Recalculate leaving_row after removing the artificial variable
                    leaving_row = find_leaving_variable(tableau, entering_col, all_slack_vars, non_basic_artificial_vars)

                    if leaving_row is None:
                        print("All ratios are infinite after removing artificial variable. Potential issue in problem formulation.")

                # If leaving_row points to the last row (which was removed), reset it
                if leaving_row is not None and leaving_row == tableau.shape[0] - 1:
                    leaving_row = None

    return tableau, num_vars, all_slack_vars, entering_col, leaving_row

In [12]:
# Define the problem
num_vars = 2
objective_coeffs = [1, 4]  # Coefficients of the objective function (minimize x1 + 4x2)
num_constraints = 3
constraint_coeffs = [[1, 1], [1, 3], [1, 2]]
constraint_types = ['>=', '>=', '<=']
b_values = [1000, 4000, 3500]

# Create the tableau
tableau, all_slack_vars = create_tableau_for_maximization(
    num_vars, objective_coeffs, num_constraints,
    constraint_coeffs, constraint_types, b_values, False  # Pass False for minimization
)

# Solve using the simplex method
optimal_solution, optimal_value = simplex_method_mixed(
    tableau, False, all_slack_vars,  # False for minimization
    objective_coeffs, constraint_coeffs, constraint_types, b_values
)

# If a solution was found, print it
if optimal_solution is not None:
    print("\nOptimal solution:")
    for i in range(num_vars):
        print(f"x{i+1} = {optimal_solution[i]}")
    print("Optimal value:", optimal_value)
else:
    print("The problem is either infeasible or unbounded.")

Original Input:
num_vars: 2
objective_coeffs: [1, 4]
num_constraints: 3
constraint_coeffs: [[1, 1], [1, 3], [1, 2]]
constraint_types: ['>=', '>=', '<=']
b_values: [1000, 4000, 3500]

After ensuring non-negative RHS:
constraint_coeffs: [[1, 1], [1, 3], [1, 2]]
constraint_types: ['>=', '>=', '<=']
b_values: [1000, 4000, 3500]

Tableau after adding slack/surplus/artificial variables:
[[1.0e+00 1.0e+00 1.0e+00 0.0e+00 0.0e+00 1.0e+03]
 [1.0e+00 3.0e+00 0.0e+00 1.0e+00 0.0e+00 4.0e+03]
 [1.0e+00 2.0e+00 0.0e+00 0.0e+00 1.0e+00 3.5e+03]
 [0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00]]
all_slack_vars: ['a1', 'a2', 's3']

Tableau after setting objective row and adding -M or +M for artificial variables:
['x1', 'x2', 'a1', 'a2', 's3', 'RHS', 'Key Column', 'Key Ratio', 'Ci', 'Bi']
['Orig', 1, 4, 0, 0, 0, 1000000.0, 1000000.0, 0]
['a1', '1/1', '1/1', '1/1', '0/1', '0/1', '1000/1', '-', '-', 'M', 'a1']
['a2', '1/1', '3/1', '0/1', '1/1', '0/1', '4000/1', '-', '-', 'M', 'a2']
['Calc', '1 - (1.0 * 

IndexError: list index out of range

In [13]:
# Define the problem
num_vars = 2
objective_coeffs = [1, 2]
num_constraints = 2
constraint_coeffs = [[1, 1], [1, 2]]
constraint_types = ['>=', '>=']
b_values = [8, 15]

# Create the tableau
tableau, all_slack_vars = create_tableau_for_maximization(
    num_vars, objective_coeffs, num_constraints,
    constraint_coeffs, constraint_types, b_values, False  # Pass False for minimization
)

# Solve using the simplex method
optimal_solution, optimal_value = simplex_method_mixed(
    tableau, False, all_slack_vars,  # False for minimization
    objective_coeffs, constraint_coeffs, constraint_types, b_values
)

# If a solution was found, print it
if optimal_solution is not None:
    print("\nOptimal solution:")
    for i in range(num_vars):
        print(f"x{i+1} = {optimal_solution[i]}")
    print("Optimal value:", optimal_value)
else:
    print("The problem is either infeasible or unbounded.")

Original Input:
num_vars: 2
objective_coeffs: [1, 2]
num_constraints: 2
constraint_coeffs: [[1, 1], [1, 2]]
constraint_types: ['>=', '>=']
b_values: [8, 15]

After ensuring non-negative RHS:
constraint_coeffs: [[1, 1], [1, 2]]
constraint_types: ['>=', '>=']
b_values: [8, 15]

Tableau after adding slack/surplus/artificial variables:
[[ 1.  1.  1.  0.  8.]
 [ 1.  2.  0.  1. 15.]
 [ 0.  0.  0.  0.  0.]]
all_slack_vars: ['a1', 'a2']

Tableau after setting objective row and adding -M or +M for artificial variables:
['x1', 'x2', 'a1', 'a2', 'RHS', 'Key Column', 'Key Ratio', 'Ci', 'Bi']
['Orig', 1, 2, 0, 0, 1000000.0, 1000000.0]
['a1', '1/1', '1/1', '1/1', '0/1', '8/1', '-', '-', 'M', 'a1']
['a2', '1/1', '2/1', '0/1', '1/1', '15/1', '-', '-', 'M', 'a2']
['Calc', '1 - (1.0 * 1000000.0 + 1.0 * 1000000.0)', '2 - (1.0 * 1000000.0 + 2.0 * 1000000.0)', '0 - (1.0 * 1000000.0 + 0.0 * 1000000.0)', '0 - (0.0 * 1000000.0 + 1.0 * 1000000.0)', '(8.0 * 1000000.0) + (15.0 * 1000000.0)']

Initial Tableau:
[[

In [14]:

# Define the problem
num_vars = 2
objective_coeffs = [12, 16]  # Coefficients of the objective function (maximize)
num_constraints = 2
constraint_coeffs = [[1, 2], [1, 1]]
constraint_types = ['>=', '>=']
b_values = [40, 30]

# It's a maximization problem
is_maximization = True

# Create the initial tableau
tableau, all_slack_vars = create_tableau_for_maximization(
    num_vars, objective_coeffs, num_constraints,
    constraint_coeffs, constraint_types, b_values, is_maximization
)

# Solve using simplex method with mixed constraints
optimal_solution, optimal_value = simplex_method_mixed(
    tableau, is_maximization, all_slack_vars,
    objective_coeffs, constraint_coeffs, constraint_types, b_values
)

Original Input:
num_vars: 2
objective_coeffs: [12, 16]
num_constraints: 2
constraint_coeffs: [[1, 2], [1, 1]]
constraint_types: ['>=', '>=']
b_values: [40, 30]

After ensuring non-negative RHS:
constraint_coeffs: [[1, 2], [1, 1]]
constraint_types: ['>=', '>=']
b_values: [40, 30]

Tableau after adding slack/surplus/artificial variables:
[[ 1.  2.  1.  0. 40.]
 [ 1.  1.  0.  1. 30.]
 [ 0.  0.  0.  0.  0.]]
all_slack_vars: ['a1', 'a2']

Tableau after setting objective row and adding -M or +M for artificial variables:
['x1', 'x2', 'a1', 'a2', 'RHS', 'Key Column', 'Key Ratio', 'Ci', 'Bi']
['Orig', 12, 16, 0, 0, 1000000.0, 1000000.0]
['a1', '1/1', '2/1', '1/1', '0/1', '40/1', '-', '-', '-M', 'a1']
['a2', '1/1', '1/1', '0/1', '1/1', '30/1', '-', '-', '-M', 'a2']
['Calc', '12 - (1.0 * 1000000.0 + 1.0 * 1000000.0)', '16 - (2.0 * 1000000.0 + 1.0 * 1000000.0)', '0 - (1.0 * 1000000.0 + 0.0 * 1000000.0)', '0 - (0.0 * 1000000.0 + 1.0 * 1000000.0)', '(40.0 * 1000000.0) + (30.0 * 1000000.0)']

Initial

In [15]:
# Define the problem
num_vars = 2
objective_coeffs = [2000, 1500]  # Coefficients of the objective function (minimize)
num_constraints = 3
constraint_coeffs = [[6, 2], [2, 4], [4, 12]]
constraint_types = ['>=', '>=', '>=']
b_values = [8, 12, 24]

# It's a maximization problem
is_maximization = False

# Create the initial tableau
tableau, all_slack_vars = create_tableau_for_maximization(
    num_vars, objective_coeffs, num_constraints,
    constraint_coeffs, constraint_types, b_values, is_maximization
)

# Solve using simplex method with mixed constraints
optimal_solution, optimal_value = simplex_method_mixed(
    tableau, is_maximization, all_slack_vars,
    objective_coeffs, constraint_coeffs, constraint_types, b_values
)


Original Input:
num_vars: 2
objective_coeffs: [2000, 1500]
num_constraints: 3
constraint_coeffs: [[6, 2], [2, 4], [4, 12]]
constraint_types: ['>=', '>=', '>=']
b_values: [8, 12, 24]

After ensuring non-negative RHS:
constraint_coeffs: [[6, 2], [2, 4], [4, 12]]
constraint_types: ['>=', '>=', '>=']
b_values: [8, 12, 24]

Tableau after adding slack/surplus/artificial variables:
[[ 6.  2.  1.  0.  0.  8.]
 [ 2.  4.  0.  1.  0. 12.]
 [ 4. 12.  0.  0.  1. 24.]
 [ 0.  0.  0.  0.  0.  0.]]
all_slack_vars: ['a1', 'a2', 'a3']

Tableau after setting objective row and adding -M or +M for artificial variables:
['x1', 'x2', 'a1', 'a2', 'a3', 'RHS', 'Key Column', 'Key Ratio', 'Ci', 'Bi']
['Orig', 2000, 1500, 0, 0, 0, 1000000.0, 1000000.0, 1000000.0]
['a1', '6/1', '2/1', '1/1', '0/1', '0/1', '8/1', '-', '-', 'M', 'a1']
['a2', '2/1', '4/1', '0/1', '1/1', '0/1', '12/1', '-', '-', 'M', 'a2']
['a3', '4/1', '12/1', '0/1', '0/1', '1/1', '24/1', '-', '-', 'M', 'a3']
['Calc', '2000 - (6.0 * 1000000.0 + 2.0 * 

In [16]:
objective_coeffs = [3, 2]  # Coefficients of the objective function
constraint_coeffs = [[1, 1], [2, 1]]  # Coefficients of the constraints
constraint_types = ['<=', '>=']  # Types of constraints
b_values = [4, 3]  # Right-hand side values of the constraints
is_maximization = True  # Whether it's a maximization problem

# Create the initial tableau
tableau, all_slack_vars = create_tableau_for_maximization(
    len(objective_coeffs), objective_coeffs, len(constraint_types),
    constraint_coeffs, constraint_types, b_values, is_maximization
)

# Run the simplex method
optimal_solution, optimal_value = simplex_method_mixed(
    tableau, is_maximization, all_slack_vars, objective_coeffs,
    constraint_coeffs, constraint_types, b_values
)

Original Input:
num_vars: 2
objective_coeffs: [3, 2]
num_constraints: 2
constraint_coeffs: [[1, 1], [2, 1]]
constraint_types: ['<=', '>=']
b_values: [4, 3]

After ensuring non-negative RHS:
constraint_coeffs: [[1, 1], [2, 1]]
constraint_types: ['<=', '>=']
b_values: [4, 3]

Tableau after adding slack/surplus/artificial variables:
[[1. 1. 1. 0. 4.]
 [2. 1. 0. 1. 3.]
 [0. 0. 0. 0. 0.]]
all_slack_vars: ['s1', 'a1']

Tableau after setting objective row and adding -M or +M for artificial variables:
['x1', 'x2', 's1', 'a1', 'RHS', 'Key Column', 'Key Ratio', 'Ci', 'Bi']
['Orig', 3, 2, 0, 0, 0, 1000000.0]
['Calc', '3 - (2.0 * 1000000.0)', '2 - (1.0 * 1000000.0)', '0 - (0.0 * 1000000.0)', '0 - (1.0 * 1000000.0)', '(3.0 * 1000000.0)']

Initial Tableau:
[[ 1.000000e+00  1.000000e+00  1.000000e+00  0.000000e+00  4.000000e+00]
 [ 2.000000e+00  1.000000e+00  0.000000e+00  1.000000e+00  3.000000e+00]
 [-1.999997e+06 -9.999980e+05  0.000000e+00 -1.000000e+06  3.000000e+06]]
['-1999997.0', '-999998.0',

IndexError: list index out of range